In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
cwd = os.getcwd()

In [4]:
df = pd.read_csv(cwd+"/data/all_matches.csv",dtype = {'match_id':'int', 'season':'str','innings':'int', 'ball':'float'},parse_dates=True)
df.season = df.season.replace({"2007/08": "2008","2009/10": "2010","2020/21":"2020"})

In [5]:
players = pd.read_csv("player_details.csv")

In [6]:
players

,Unnamed: 0,season,team,identifier,name,unique_name,key_cricinfo,batting_style,player_type,bowling_style
0,0,2007/08,Kolkata Knight Riders,725529bc,SC Ganguly,SC Ganguly,28779,Left hand Bat,Batter,Right arm Medium
1,1,2007/08,Kolkata Knight Riders,b8a55852,BB McCullum,BB McCullum,37737,Right hand Bat,Wicketkeeper Batter,NaN
2,2,2007/08,Kolkata Knight Riders,7d415ea5,RT Ponting,RT Ponting,7133,Right hand Bat,Top order Batter,Right arm Medium
3,3,2007/08,Kolkata Knight Riders,fd835ab3,DJ Hussey,DJ Hussey,5766,Right hand Bat,Allrounder,Right arm Offbreak
4,4,2007/08,Kolkata Knight Riders,9ab63e7b,Mohammad Hafeez,Mohammad Hafeez,41434,Right hand Bat,Allrounder,Right arm Offbreak
...,...,...,...,...,...,...,...,...,...,...
2493,2493,2022,Rajasthan Royals,944533a5,KK Nair,KK Nair,398439,Right hand Bat,Top order Batter,Right arm Offbreak
2494,2494,2022,Rajasthan Royals,529eb9e0,OC McCoy,OC McCoy,906783,Left hand Bat,Bowler,Left arm Fast medium
2495,2495,2022,Punjab Kings,9eb1455b,NT Ellis,NT Ellis,826915,Right hand Bat,Bowler,Right arm Fast medium
2496,2496,2022,Mumbai Indians,0c9652b0,HR Shokeen,HR Shokeen,1175423,Right hand Bat,Bowler,Right arm Offbreak


In [7]:
consolidated_df = pd.DataFrame()

In [8]:
consolidated_df = df.groupby(["striker"])["runs_off_bat"].agg(balls_faced="count",runs_total="sum").reset_index()

In [9]:
consolidated_df[consolidated_df["striker"]=="SK Raina"]

,striker,balls_faced,runs_total
479,SK Raina,4177,5536


In [10]:
consolidated_df

,striker,balls_faced,runs_total
0,A Ashish Reddy,196,280
1,A Badoni,92,120
2,A Chandila,7,4
3,A Chopra,75,53
4,A Choudhary,20,25
...,...,...,...
582,YV Takawale,183,192
583,Yashpal Singh,67,47
584,Younis Khan,7,3
585,Yuvraj Singh,2207,2754


In [25]:
cons_df_0_6 = df[df['ball']<=6].groupby(["striker"])["runs_off_bat"].agg(balls_0to6="count",runs_0to6="sum").reset_index()

In [26]:
cons_df_7_15 = df[(df['ball']>6) & (df['ball']<=15)].groupby(["striker"])["runs_off_bat"].agg(balls_7to15="count",runs_7to15="sum").reset_index()

In [27]:
cons_df_death = df[(df['ball']>15)].groupby(["striker"])["runs_off_bat"].agg(balls_death="count",runs_death="sum").reset_index()

In [34]:
consolidated_df = consolidated_df.merge(cons_df_0_6,on='striker',how='left')
consolidated_df = consolidated_df.merge(cons_df_7_15,on='striker',how='left')
consolidated_df = consolidated_df.merge(cons_df_death,on='striker',how='left')

In [36]:
consolidated_df.fillna(0,inplace=True)

In [37]:
consolidated_df

,striker,balls_faced,runs_total,balls_0to6,runs_0to6,balls_7to15,runs_7to15,balls_death,runs_death
0,A Ashish Reddy,196,280,7.0,5.0,41.0,61.0,148.0,214.0
1,A Badoni,92,120,6.0,1.0,37.0,42.0,49.0,77.0
2,A Chandila,7,4,0.0,0.0,0.0,0.0,7.0,4.0
3,A Chopra,75,53,41.0,27.0,32.0,25.0,2.0,1.0
4,A Choudhary,20,25,0.0,0.0,0.0,0.0,20.0,25.0
...,...,...,...,...,...,...,...,...,...
582,YV Takawale,183,192,137.0,121.0,33.0,42.0,13.0,29.0
583,Yashpal Singh,67,47,37.0,17.0,17.0,13.0,13.0,17.0
584,Younis Khan,7,3,7.0,3.0,0.0,0.0,0.0,0.0
585,Yuvraj Singh,2207,2754,197.0,193.0,1475.0,1655.0,535.0,906.0


In [43]:
consolidated_df["sr_0to6"] = consolidated_df["runs_0to6"]*100/consolidated_df["balls_0to6"]
consolidated_df["sr_7to15"] = consolidated_df["runs_7to15"]*100/consolidated_df["balls_7to15"]
consolidated_df["sr_death"] = consolidated_df["runs_death"]*100/consolidated_df["balls_death"]

In [44]:
consolidated_df.fillna(0,inplace=True)

In [46]:
consolidated_df["0to6_bat_salience"] = consolidated_df["balls_0to6"]*100/consolidated_df["balls_faced"]
consolidated_df["7to15_bat_salience"] = consolidated_df["balls_7to15"]*100/consolidated_df["balls_faced"]
consolidated_df["death_bat_salience"] = consolidated_df["balls_death"]*100/consolidated_df["balls_faced"]

In [47]:
consolidated_df

,striker,balls_faced,runs_total,balls_0to6,runs_0to6,balls_7to15,runs_7to15,balls_death,runs_death,sr_0to6,sr_7to15,sr_death,0to6_bat_salience,7to15_bat_salience,death_bat_salience
0,A Ashish Reddy,196,280,7.0,5.0,41.0,61.0,148.0,214.0,71.428571,148.780488,144.594595,3.571429,20.918367,75.510204
1,A Badoni,92,120,6.0,1.0,37.0,42.0,49.0,77.0,16.666667,113.513514,157.142857,6.521739,40.217391,53.260870
2,A Chandila,7,4,0.0,0.0,0.0,0.0,7.0,4.0,0.000000,0.000000,57.142857,0.000000,0.000000,100.000000
3,A Chopra,75,53,41.0,27.0,32.0,25.0,2.0,1.0,65.853659,78.125000,50.000000,54.666667,42.666667,2.666667
4,A Choudhary,20,25,0.0,0.0,0.0,0.0,20.0,25.0,0.000000,0.000000,125.000000,0.000000,0.000000,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,YV Takawale,183,192,137.0,121.0,33.0,42.0,13.0,29.0,88.321168,127.272727,223.076923,74.863388,18.032787,7.103825
583,Yashpal Singh,67,47,37.0,17.0,17.0,13.0,13.0,17.0,45.945946,76.470588,130.769231,55.223881,25.373134,19.402985
584,Younis Khan,7,3,7.0,3.0,0.0,0.0,0.0,0.0,42.857143,0.000000,0.000000,100.000000,0.000000,0.000000
585,Yuvraj Singh,2207,2754,197.0,193.0,1475.0,1655.0,535.0,906.0,97.969543,112.203390,169.345794,8.926144,66.832805,24.241051


In [122]:
#ref to create and store named df from a list : https://stackoverflow.com/questions/67576606/create-multiple-empty-dataframes-named-from-a-list-using-a-loop

batsman_h2h_dict = {}
batsman_style_dict = {}
for batsman in consolidated_df["striker"]:
#for batsman in ["SK Raina"]: #for debugging
    df_batsman = df[df['striker'] == batsman]
    #print(df_batsman.shape)
    bat_h2h = df_batsman.groupby(['bowler'])["runs_off_bat"].agg(balls_faced="count",runs_total="sum").reset_index()
    bat_h2h["dismissals"] = df_batsman.groupby(['bowler'])["player_dismissed"].agg(balls_faced="count",dismissals="count").reset_index()["dismissals"]
    bat_h2h.sort_values(by = 'balls_faced',inplace=True,ascending=False)
    bat_h2h.rename({'bowler':'unique_name'},axis=1,inplace=True)
    bat_h2h_bowl = bat_h2h.merge(players,how='left',on='unique_name').drop(columns=['team','identifier','key_cricinfo','batting_style','name','player_type','season','Unnamed: 0'])
    bat_h2h_bowl.drop_duplicates(inplace=True)
    bat_h2h_bowl["strike_rate"] = (bat_h2h_bowl["runs_total"]*100/bat_h2h_bowl["balls_faced"])
    bat_h2h_bowl["balls_per_out"] = (bat_h2h_bowl["balls_faced"]/bat_h2h_bowl["dismissals"])
    bat_h2h_bowl = bat_h2h_bowl.round({'strike_rate':2,'balls_per_out':1})
    bat_h2h_style = bat_h2h_bowl.groupby(['bowling_style'])["balls_faced"].agg(balls_faced="sum").reset_index()
    bat_h2h_style["runs_total"] = bat_h2h_bowl.groupby(['bowling_style'])["runs_total"].agg(runs_total="sum").reset_index()["runs_total"]
    bat_h2h_style["dismissals"] = bat_h2h_bowl.groupby(['bowling_style'])["dismissals"].agg(dismissals="sum").reset_index()["dismissals"]
    bat_h2h_style.sort_values(by = 'balls_faced',inplace=True,ascending=False)
    bat_h2h_style["strike_rate"] = (bat_h2h_style["runs_total"]*100/bat_h2h_style["balls_faced"])
    bat_h2h_style["balls_per_out"] = (bat_h2h_style["balls_faced"]/bat_h2h_style["dismissals"])
    bat_h2h_style = bat_h2h_style.round({'strike_rate':2,'balls_per_out':1})
    
    batsman_h2h_dict[batsman] = bat_h2h_bowl
    batsman_style_dict[batsman] = bat_h2h_style
    

In [123]:
#ref to create and store named df from a list : https://stackoverflow.com/questions/67576606/create-multiple-empty-dataframes-named-from-a-list-using-a-loop

batsman_h2h_dict_0to6 = {}
batsman_style_dict_0to6 = {}
for batsman in consolidated_df["striker"]:
#for batsman in ["SK Raina"]: #for debugging
    df_0to6 = df[df['ball']<=6]
    df_batsman = df_0to6[df_0to6['striker'] == batsman]
    #print(df_batsman.shape)
    bat_h2h = df_batsman.groupby(['bowler'])["runs_off_bat"].agg(balls_faced="count",runs_total="sum").reset_index()
    bat_h2h["dismissals"] = df_batsman.groupby(['bowler'])["player_dismissed"].agg(balls_faced="count",dismissals="count").reset_index()["dismissals"]
    bat_h2h.sort_values(by = 'balls_faced',inplace=True,ascending=False)
    bat_h2h.rename({'bowler':'unique_name'},axis=1,inplace=True)
    bat_h2h_bowl = bat_h2h.merge(players,how='left',on='unique_name').drop(columns=['team','identifier','key_cricinfo','batting_style','name','player_type','season','Unnamed: 0'])
    bat_h2h_bowl.drop_duplicates(inplace=True)
    bat_h2h_bowl["strike_rate"] = (bat_h2h_bowl["runs_total"]*100/bat_h2h_bowl["balls_faced"])
    bat_h2h_bowl["balls_per_out"] = (bat_h2h_bowl["balls_faced"]/bat_h2h_bowl["dismissals"])
    bat_h2h_bowl = bat_h2h_bowl.round({'strike_rate':2,'balls_per_out':1})
    bat_h2h_style = bat_h2h_bowl.groupby(['bowling_style'])["balls_faced"].agg(balls_faced="sum").reset_index()
    bat_h2h_style["runs_total"] = bat_h2h_bowl.groupby(['bowling_style'])["runs_total"].agg(runs_total="sum").reset_index()["runs_total"]
    bat_h2h_style["dismissals"] = bat_h2h_bowl.groupby(['bowling_style'])["dismissals"].agg(dismissals="sum").reset_index()["dismissals"]
    bat_h2h_style.sort_values(by = 'balls_faced',inplace=True,ascending=False)
    bat_h2h_style["strike_rate"] = (bat_h2h_style["runs_total"]*100/bat_h2h_style["balls_faced"])
    bat_h2h_style["balls_per_out"] = (bat_h2h_style["balls_faced"]/bat_h2h_style["dismissals"])
    bat_h2h_style = bat_h2h_style.round({'strike_rate':2,'balls_per_out':1})
    
    batsman_h2h_dict_0to6[batsman] = bat_h2h_bowl
    batsman_style_dict_0to6[batsman] = bat_h2h_style
    

In [124]:
#ref to create and store named df from a list : https://stackoverflow.com/questions/67576606/create-multiple-empty-dataframes-named-from-a-list-using-a-loop

batsman_h2h_dict_7to15 = {}
batsman_style_dict_7to15 = {}
for batsman in consolidated_df["striker"]:
#for batsman in ["SK Raina"]: #for debugging
    df_7to15 = df[(df['ball']>6) & (df['ball']<=15)]
    df_batsman = df_7to15[df_7to15['striker'] == batsman]
    #print(df_batsman.shape)
    bat_h2h = df_batsman.groupby(['bowler'])["runs_off_bat"].agg(balls_faced="count",runs_total="sum").reset_index()
    bat_h2h["dismissals"] = df_batsman.groupby(['bowler'])["player_dismissed"].agg(balls_faced="count",dismissals="count").reset_index()["dismissals"]
    bat_h2h.sort_values(by = 'balls_faced',inplace=True,ascending=False)
    bat_h2h.rename({'bowler':'unique_name'},axis=1,inplace=True)
    bat_h2h_bowl = bat_h2h.merge(players,how='left',on='unique_name').drop(columns=['team','identifier','key_cricinfo','batting_style','name','player_type','season','Unnamed: 0'])
    bat_h2h_bowl.drop_duplicates(inplace=True)
    bat_h2h_bowl["strike_rate"] = (bat_h2h_bowl["runs_total"]*100/bat_h2h_bowl["balls_faced"])
    bat_h2h_bowl["balls_per_out"] = (bat_h2h_bowl["balls_faced"]/bat_h2h_bowl["dismissals"])
    bat_h2h_bowl = bat_h2h_bowl.round({'strike_rate':2,'balls_per_out':1})
    bat_h2h_style = bat_h2h_bowl.groupby(['bowling_style'])["balls_faced"].agg(balls_faced="sum").reset_index()
    bat_h2h_style["runs_total"] = bat_h2h_bowl.groupby(['bowling_style'])["runs_total"].agg(runs_total="sum").reset_index()["runs_total"]
    bat_h2h_style["dismissals"] = bat_h2h_bowl.groupby(['bowling_style'])["dismissals"].agg(dismissals="sum").reset_index()["dismissals"]
    bat_h2h_style.sort_values(by = 'balls_faced',inplace=True,ascending=False)
    bat_h2h_style["strike_rate"] = (bat_h2h_style["runs_total"]*100/bat_h2h_style["balls_faced"])
    bat_h2h_style["balls_per_out"] = (bat_h2h_style["balls_faced"]/bat_h2h_style["dismissals"])
    bat_h2h_style = bat_h2h_style.round({'strike_rate':2,'balls_per_out':1})
    
    batsman_h2h_dict_7to15[batsman] = bat_h2h_bowl
    batsman_style_dict_7to15[batsman] = bat_h2h_style
    

In [125]:
#ref to create and store named df from a list : https://stackoverflow.com/questions/67576606/create-multiple-empty-dataframes-named-from-a-list-using-a-loop

batsman_h2h_dict_death = {}
batsman_style_dict_death = {}
for batsman in consolidated_df["striker"]:
#for batsman in ["SK Raina"]: #for debugging
    df_death = df[(df['ball']>15)]
    df_batsman = df_death[df_death['striker'] == batsman]
    #print(df_batsman.shape)
    bat_h2h = df_batsman.groupby(['bowler'])["runs_off_bat"].agg(balls_faced="count",runs_total="sum").reset_index()
    bat_h2h["dismissals"] = df_batsman.groupby(['bowler'])["player_dismissed"].agg(balls_faced="count",dismissals="count").reset_index()["dismissals"]
    bat_h2h.sort_values(by = 'balls_faced',inplace=True,ascending=False)
    bat_h2h.rename({'bowler':'unique_name'},axis=1,inplace=True)
    bat_h2h_bowl = bat_h2h.merge(players,how='left',on='unique_name').drop(columns=['team','identifier','key_cricinfo','batting_style','name','player_type','season','Unnamed: 0'])
    bat_h2h_bowl.drop_duplicates(inplace=True)
    bat_h2h_bowl["strike_rate"] = (bat_h2h_bowl["runs_total"]*100/bat_h2h_bowl["balls_faced"])
    bat_h2h_bowl["balls_per_out"] = (bat_h2h_bowl["balls_faced"]/bat_h2h_bowl["dismissals"])
    bat_h2h_bowl = bat_h2h_bowl.round({'strike_rate':2,'balls_per_out':1})
    bat_h2h_style = bat_h2h_bowl.groupby(['bowling_style'])["balls_faced"].agg(balls_faced="sum").reset_index()
    bat_h2h_style["runs_total"] = bat_h2h_bowl.groupby(['bowling_style'])["runs_total"].agg(runs_total="sum").reset_index()["runs_total"]
    bat_h2h_style["dismissals"] = bat_h2h_bowl.groupby(['bowling_style'])["dismissals"].agg(dismissals="sum").reset_index()["dismissals"]
    bat_h2h_style.sort_values(by = 'balls_faced',inplace=True,ascending=False)
    bat_h2h_style["strike_rate"] = (bat_h2h_style["runs_total"]*100/bat_h2h_style["balls_faced"])
    bat_h2h_style["balls_per_out"] = (bat_h2h_style["balls_faced"]/bat_h2h_style["dismissals"])
    bat_h2h_style = bat_h2h_style.round({'strike_rate':2,'balls_per_out':1})
    
    batsman_h2h_dict_death[batsman] = bat_h2h_bowl
    batsman_style_dict_death[batsman] = bat_h2h_style
    

In [127]:
batsman_style_dict["MS Dhoni"]

""


In [128]:
batsman_style_dict_0to6["MS Dhoni"]

,bowling_style,balls_faced,runs_total,dismissals,strike_rate,balls_per_out
6,Right arm Medium,22,17,0,77.27,inf
1,Left arm Fast medium,12,13,0,108.33,inf
4,Right arm Fast,12,15,1,125.00,12.0
5,Right arm Fast medium,11,6,0,54.55,inf
9,Slow Left arm Orthodox,7,1,0,14.29,inf
0,Left arm Fast,2,0,1,0.00,2.0
7,Right arm Medium fast,2,0,0,0.00,inf
2,Left arm Medium,1,1,0,100.00,inf
3,Left arm Wrist spin,1,0,0,0.00,inf
8,"Right arm Medium, Right arm Offbreak",1,0,0,0.00,inf


In [129]:
batsman_style_dict_7to15["MS Dhoni"]

,bowling_style,balls_faced,runs_total,dismissals,strike_rate,balls_per_out
14,Slow Left arm Orthodox,386,360,9,93.26,42.9
9,Right arm Medium,264,297,11,112.50,24.0
12,Right arm Offbreak,228,246,2,107.89,114.0
6,Legbreak Googly,210,153,7,72.86,30.0
5,Legbreak,173,177,5,102.31,34.6
8,Right arm Fast medium,157,141,1,89.81,157.0
10,Right arm Medium fast,130,146,2,112.31,65.0
7,Right arm Fast,127,96,2,75.59,63.5
1,Left arm Fast medium,56,68,1,121.43,56.0
3,Left arm Medium fast,40,55,1,137.50,40.0


In [130]:
batsman_h2h_dict_death["MS Dhoni"]

,unique_name,balls_faced,runs_total,dismissals,bowling_style,strike_rate,balls_per_out
0,B Kumar,56,94,0,Right arm Medium,167.86,inf
12,SP Narine,45,26,1,Right arm Offbreak,57.78,45.0
23,JJ Bumrah,42,43,4,Right arm Fast,102.38,10.5
33,SL Malinga,41,68,2,Right arm Fast,165.85,20.5
42,Z Khan,38,63,6,Left arm Fast medium,165.79,6.3
...,...,...,...,...,...,...,...
997,LH Ferguson,1,0,0,Right arm Fast,0.00,inf
1002,AN Ahmed,1,0,1,Right arm Medium fast,0.00,1.0
1008,M Jansen,1,0,1,Left arm Fast,0.00,1.0
1010,BA Bhatt,1,1,0,Slow Left arm Orthodox,100.00,inf


In [132]:
batsman_h2h_dict["CA Pujara"]

""


In [133]:
batsman_style_dict["CA Pujara"]

""


In [134]:
batsman_style_dict_0to6["CA Pujara"]

,bowling_style,balls_faced,runs_total,dismissals,strike_rate,balls_per_out
7,Right arm Medium,55,48,2,87.27,27.5
5,Right arm Fast,46,52,1,113.04,46.0
2,Left arm Medium fast,32,33,1,103.12,32.0
1,Left arm Fast medium,27,24,1,88.89,27.0
6,Right arm Fast medium,24,16,1,66.67,24.0
8,Right arm Medium fast,22,22,1,100.00,22.0
0,Left arm Fast,10,3,0,30.00,inf
10,Right arm Offbreak,9,4,2,44.44,4.5
9,"Right arm Medium, Right arm Offbreak",6,9,1,150.00,6.0
4,Legbreak Googly,3,4,0,133.33,inf


In [135]:
batsman_style_dict_7to15["CA Pujara"]

,bowling_style,balls_faced,runs_total,dismissals,strike_rate,balls_per_out
6,Right arm Medium,37,40,2,108.11,18.5
2,Legbreak,30,29,0,96.67,inf
3,Legbreak Googly,18,19,0,105.56,inf
9,Slow Left arm Orthodox,12,13,0,108.33,inf
8,Right arm Offbreak,8,9,1,112.50,8.0
7,Right arm Medium fast,7,6,0,85.71,inf
5,Right arm Fast medium,5,8,0,160.00,inf
4,Right arm Fast,4,1,0,25.00,inf
1,Left arm Medium fast,2,2,0,100.00,inf
0,Left arm Fast medium,1,1,0,100.00,inf


In [136]:
batsman_h2h_dict_death["CA Pujara"]

,unique_name,balls_faced,runs_total,dismissals,bowling_style,strike_rate,balls_per_out
0,DW Steyn,13,12,1,Right arm Fast,92.31,13.0
11,JH Kallis,6,14,1,Right arm Fast medium,233.33,6.0
18,A Kumble,5,2,0,Legbreak Googly,40.00,inf
21,JP Faulkner,4,9,0,Left arm Fast medium,225.00,inf
28,PV Tambe,3,3,0,Legbreak,100.00,inf
32,R Ashwin,3,1,0,Right arm Offbreak,33.33,inf
45,S Randiv,2,1,1,Right arm Offbreak,50.00,2.0
46,A Mishra,1,1,0,Legbreak,100.00,inf
60,AJ Finch,1,1,0,Slow Left arm Orthodox,100.00,inf
71,DE Bollinger,1,0,1,Left arm Fast medium,0.00,1.0
